In [ ]:
import os
import sys
import time
import argparse
import re
import logging
from dotenv import load_dotenv

from openai import AzureOpenAI
import tiktoken
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility
import numpy as np
import fitz  # PyMuPDF
from typing import List, Tuple

import nltk
from langchain.text_splitter import RecursiveCharacterTextSplitter  # Thêm import LangChain

# ---------------------------------------
# Thiết lập logging
logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(asctime)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Kiểm tra và tải tokenizer punkt của NLTK nếu cần
try:
    nltk.data.find('tokenizers/punkt')
    logging.info("NLTK punkt đã có sẵn.")
except LookupError:
    logging.info("Đang tải NLTK punkt...")
    nltk.download('punkt', quiet=True)
    logging.info("NLTK punkt tải xong.")

# ---------------------------------------
# 1. Load config Azure OpenAI và Milvus từ .env
load_dotenv()

AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")  # ví dụ "2024-10-21"
EMBED_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT_EMBED")
CHAT_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT_CHAT")

if not all([AZURE_OPENAI_ENDPOINT, AZURE_OPENAI_KEY, AZURE_OPENAI_API_VERSION, EMBED_DEPLOYMENT, CHAT_DEPLOYMENT]):
    logging.error("Vui lòng kiểm tra biến môi trường Azure OpenAI trong .env")
    sys.exit(1)

# Khởi tạo Azure OpenAI client
client = AzureOpenAI(
    api_key=AZURE_OPENAI_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Milvus config
MILVUS_HOST = os.getenv("MILVUS_HOST", "localhost")
MILVUS_PORT = int(os.getenv("MILVUS_PORT", 19530))

# ---------------------------------------
# 2. Kết nối Milvus và tạo Collection nếu cần
def connect_milvus():
    connections.connect("default", host=MILVUS_HOST, port=MILVUS_PORT)
    logging.info(f"[Milvus] Connected to {MILVUS_HOST}:{MILVUS_PORT}")

def create_collection(collection_name: str, dim: int):
    existing = utility.list_collections()
    if collection_name in existing:
        logging.info(f"[Milvus] Collection '{collection_name}' đã tồn tại, drop để tạo lại mới.")
        Collection(collection_name).drop()
    # Định nghĩa schema
    fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
        FieldSchema(name="doc_id", dtype=DataType.INT64),
        FieldSchema(name="chunk_id", dtype=DataType.INT64),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535)
    ]
    schema = CollectionSchema(fields, description="RAG internal knowledge base")
    collection = Collection(name=collection_name, schema=schema)
    # Tạo index cho vector field
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "IP",           # dùng IP để tìm cosine similarity khi vector đã normalize
        "params": {"nlist": 128}
    }
    logging.info(f"[Milvus] Creating index on '{collection_name}.embedding' ...")
    collection.create_index(field_name="embedding", index_params=index_params)
    collection.load()
    logging.info(f"[Milvus] Collection '{collection_name}' is ready.")
    return collection

# ---------------------------------------
# 3. Đọc tài liệu nội bộ
def load_texts_from_pdf(path: str) -> str:
    """Đọc text và table từ PDF qua PyMuPDF, table sẽ được prefix [TABLE]."""
    try:
        doc = fitz.open(path)
    except Exception as e:
        logging.warning(f"Không mở được PDF '{path}': {e}")
        return ""
    blocks = []
    for page in doc:
        for b in page.get_text("blocks"):
            text = b[4].strip()
            if not text:
                continue
            # Nhận diện table đơn giản: có tab hoặc nhiều khoảng trắng liên tiếp
            if "\t" in text or re.search(r' {2,}', text):
                blocks.append("[TABLE]\n" + text)
            else:
                blocks.append(text)
    return "\n\n".join(blocks)

def load_text_from_txt(path: str) -> str:
    """Đọc text thuần từ file .txt."""
    try:
        with open(path, encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        logging.warning(f"Không đọc được TXT '{path}': {e}")
        return ""

def load_documents_from_folder(folder: str) -> List[Tuple[int, str]]:
    """
    Quét thư mục, đọc các file .pdf và .txt.
    Trả về list các tuple (doc_id, text_content). doc_id: 0,1,2,...
    """
    docs = []
    doc_id = 0
    for root, dirs, files in os.walk(folder):
        for fname in files:
            path = os.path.join(root, fname)
            text = ""
            if fname.lower().endswith(".pdf"):
                text = load_texts_from_pdf(path)
            elif fname.lower().endswith(".txt"):
                text = load_text_from_txt(path)
            else:
                continue
            if text and text.strip():
                docs.append((doc_id, text))
                logging.info(f"[Load] doc_id={doc_id}, file='{path}', length={len(text)} chars")
                doc_id += 1
            else:
                logging.info(f"[Skip] file '{path}' không có nội dung hoặc đọc lỗi.")
    if not docs:
        logging.warning("[Warning] Không tìm thấy tài liệu hợp lệ trong thư mục.")
    return docs

# ---------------------------------------
# 4. Chunking cải tiến
# Sử dụng tiktoken để đếm token
_enc = None

def count_tokens(text: str) -> int:
    global _enc
    if _enc is None:
        try:
            _enc = tiktoken.get_encoding("cl100k_base")
        except Exception:
            try:
                _enc = tiktoken.encoding_for_model("gpt-3.5-turbo")
            except Exception:
                _enc = tiktoken.get_encoding("cl100k_base")
    try:
        return len(_enc.encode(text))
    except Exception:
        return len(text.split())

def chunk_text_langchain(text: str, max_tokens: int = 500, overlap_tokens: int = 50) -> list:
    """
    Chunking sử dụng LangChain RecursiveCharacterTextSplitter dựa trên số token.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_tokens,
        chunk_overlap=overlap_tokens,
        length_function=count_tokens
    )
    return splitter.split_text(text)

def chunk_text_langchain_with_table(text: str, max_tokens: int = 500, overlap_tokens: int = 50) -> list:
    """
    Chunking sử dụng LangChain, nhưng bảng ([TABLE]) sẽ được chunk riêng biệt.
    - Nếu là block bảng: chunk từng bảng riêng (có thể chia nhỏ theo dòng hoặc theo token)
    - Nếu là block thường: chunk như cũ
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_tokens,
        chunk_overlap=overlap_tokens,
        length_function=count_tokens
    )
    chunks = []
    blocks = text.split('\n\n')
    for block in blocks:
        block = block.strip()
        if not block:
            continue
        if block.startswith('[TABLE]'):
            # Chunk bảng: chia nhỏ theo dòng hoặc theo token
            table_content = block[len('[TABLE]'):].strip()
            # Có thể chia theo dòng hoặc chunk theo token, ở đây chunk theo token
            table_chunks = splitter.split_text(table_content)
            # Gắn lại prefix [TABLE] cho mỗi chunk bảng
            for tchunk in table_chunks:
                chunks.append('[TABLE]\n' + tchunk)
        else:
            # Chunk đoạn thường
            chunks.extend(splitter.split_text(block))
    return chunks

# ---------------------------------------
# 5. Embedding với Azure OpenAI
def get_embeddings(texts: List[str], batch_size: int = 20) -> List[List[float]]:
    embeddings: List[List[float]] = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        try:
            responses = client.embeddings.create(
                model=EMBED_DEPLOYMENT,
                input=batch
            )
            for data in responses.data:
                embeddings.append(data.embedding)
        except Exception as e:
            logging.warning(f"Lấy embedding batch lỗi: {e}. Thử lại sau 5s.")
            time.sleep(5)
            responses = client.embeddings.create(
                model=EMBED_DEPLOYMENT,
                input=batch
            )
            for data in responses.data:
                embeddings.append(data.embedding)
        time.sleep(0.1)
    return embeddings

def normalize_vector(vec: List[float]) -> List[float]:
    arr = np.array(vec, dtype=np.float32)
    norm = np.linalg.norm(arr)
    if norm == 0:
        return arr.tolist()
    return (arr / norm).tolist()

def insert_chunks_to_milvus(collection: Collection, doc_id: int, chunks: List[str], batch_size: int = 20):
    n = len(chunks)
    chunk_ids = list(range(n))
    for i in range(0, n, batch_size):
        batch_chunks = chunks[i:i+batch_size]
        batch_chunk_ids = chunk_ids[i:i+batch_size]
        embs = get_embeddings(batch_chunks, batch_size=batch_size)
        embs_norm = [normalize_vector(e) for e in embs]
        doc_ids = [doc_id] * len(batch_chunks)
        texts = batch_chunks
        # Order: embedding, doc_id, chunk_id, text
        entities = [
            embs_norm,
            doc_ids,
            batch_chunk_ids,
            texts
        ]
        try:
            collection.insert(entities)
        except Exception as e:
            logging.error(f"Insert vào Milvus lỗi: {e}")
        time.sleep(0.05)
    collection.flush()

# ---------------------------------------
# 6. Search và chat
def embed_query(query: str) -> List[float]:
    try:
        response = client.embeddings.create(
            model=EMBED_DEPLOYMENT,
            input=[query]
        )
    except Exception as e:
        logging.warning(f"embed_query lỗi: {e}. Thử lại sau 2s.")
        time.sleep(2)
        response = client.embeddings.create(
            model=EMBED_DEPLOYMENT,
            input=[query]
        )
    vec = response.data[0].embedding
    return normalize_vector(vec)

def search_milvus(collection: Collection, query_embedding: List[float], top_k: int = 30) -> List[Tuple[str, float]]:
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param={"metric_type": "IP", "params": {"nprobe": 10}},
        limit=top_k,
        output_fields=["doc_id", "chunk_id", "text"]
    )
    hits = results[0]
    contexts: List[Tuple[str, float]] = []
    for hit in hits:
        txt = hit.entity.get("text")
        score = hit.score
        contexts.append((txt, score))
    return contexts

def build_prompt(contexts: List[Tuple[str, float]], question: str, max_context_tokens: int = 1500) -> List[dict]:
    contexts_sorted = sorted(contexts, key=lambda x: x[1], reverse=True)
    try:
        encoder = tiktoken.get_encoding("cl100k_base")
    except Exception:
        encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")
    selected = []
    total_tokens = 0
    for text, score in contexts_sorted:
        toks = len(encoder.encode(text))
        if total_tokens + toks > max_context_tokens:
            continue
        selected.append(text)
        total_tokens += toks
    context_text = "\n---\n".join(selected) if selected else ""
    system_message = {
        "role": "system",
        "content": (
            "Bạn là trợ lý nội bộ. Sử dụng chỉ thông tin nội bộ được cung cấp ở phần 'context' nếu có liên quan "
            "để trả lời câu hỏi. Nếu không tìm thấy thông tin đủ, hãy trả lời trung thực là không biết."
        )
    }
    if context_text:
        user_content = f"Dữ liệu tham khảo:\n{context_text}\n\nCâu hỏi: {question}"
    else:
        user_content = f"Câu hỏi: {question}\n(Lưu ý: Không tìm thấy thông tin liên quan trong knowledge base.)"
    user_message = {"role": "user", "content": user_content}
    return [system_message, user_message]

def get_answer_from_azure(messages: List[dict], max_tokens: int = 512, temperature: float = 0.2) -> str:
    try:
        response = client.chat.completions.create(
            model=CHAT_DEPLOYMENT,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
    except Exception as e:
        logging.warning(f"ChatCompletion lỗi: {e}. Thử lại sau 2s.")
        time.sleep(2)
        response = client.chat.completions.create(
            model=CHAT_DEPLOYMENT,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
    answer = response.choices[0].message.content
    return answer

def answer_query(collection: Collection, question: str, top_k: int = 30) -> str:
    q_emb = embed_query(question)
    contexts = search_milvus(collection, q_emb, top_k=top_k)
    low = question.strip().lower()
    greetings = {"hi", "hello", "chào", "xin chào", "hey"}
    if low in greetings or any(low.startswith(g) for g in greetings):
        return "Chào bạn! Bạn có thể hỏi tôi điều gì về nội dung đã được cung cấp."
    messages = build_prompt(contexts, question)
    answer = get_answer_from_azure(messages)
    return answer

# ---------------------------------------
# 7. CLI hoặc API
def interactive_cli(collection: Collection):
    logging.info("=== ChatBot RAG (Azure OpenAI + Milvus) ===")
    logging.info("Nhập câu hỏi của bạn (gõ 'exit' hoặc 'quit' để dừng):")
    # chat_history = []  # Lưu lịch sử hội thoại
    # max_history = 10   # Số lượt hội thoại gần nhất giữ lại (có thể điều chỉnh)
    while True:
        try:
            question = input("Bạn hỏi: ").strip()
        except (KeyboardInterrupt, EOFError):
            logging.info("Thoát.")
            break
        if not question:
            continue
        if question.lower() in {"exit", "quit"}:
            logging.info("Thoát.")
            break
        # Lấy context từ Milvus như cũ
        q_emb = embed_query(question)
        contexts = search_milvus(collection, q_emb, top_k=30)
        messages = build_prompt(contexts, question)
        # Nối lịch sử hội thoại vào sau messages (trừ câu hỏi hiện tại)
        # Chỉ lấy max_history lượt gần nhất để tránh quá dài
        # trimmed_history = chat_history[-max_history*2:]  # mỗi lượt gồm user+assistant
        # full_messages = messages + trimmed_history
        ans = get_answer_from_azure(messages)
        print("Bot trả lời:", ans)
        print("-" * 40)
        # Lưu vào lịch sử
        # chat_history.append({"role": "user", "content": question})
        # chat_history.append({"role": "assistant", "content": ans})

def main():
    parser = argparse.ArgumentParser(description="RAG ChatBot with Azure OpenAI + Milvus")
    parser.add_argument(
        "--mode",
        choices=["index", "chat", "index_and_chat"],
        default="index_and_chat",
        help="Chế độ: 'index' chỉ indexing, 'chat' chỉ chat (giả định đã index), 'index_and_chat' làm cả hai."
    )
    parser.add_argument(
        "--docs_folder",
        type=str,
        default="./docs",
        help="Thư mục chứa tài liệu để index (pdf, txt)."
    )
    parser.add_argument(
        "--collection_name",
        type=str,
        default="docsEngLC21",
        help="Tên collection Milvus."
    )
    parser.add_argument(
        "--dim",
        type=int,
        default=1536,
        help="Dimension embedding (ví dụ 1536 cho text-embedding-ada-002)."
    )
    parser.add_argument(
        "--top_k",
        type=int,
        default=30,
        help="Số top chunks lấy từ Milvus khi search."
    )
    parser.add_argument(
        "--max_tokens",
        type=int,
        default=500,
        help="Giới hạn token mỗi chunk khi indexing."
    )
    parser.add_argument(
        "--overlap_tokens",
        type=int,
        default=50,
        help="Số token overlap giữa các chunk khi indexing."
    )
    parser.add_argument(
        "--verbose_chunk",
        action="store_true",
        help="Nếu bật, sẽ in thông tin chi tiết khi chunking."
    )
    args = parser.parse_args()

    connect_milvus()

    collection = None
    if args.mode in {"index", "index_and_chat"}:
        collection = create_collection(args.collection_name, args.dim)
        docs = load_documents_from_folder(args.docs_folder)
        if not docs:
            logging.error("Không có tài liệu để index. Kiểm tra lại thư mục.")
            sys.exit(1)
        for doc_id, text in docs:
            logging.info(f"[Indexing] doc_id={doc_id}, chunking ...")
            chunks = chunk_text_langchain_with_table(
                text,
                max_tokens=args.max_tokens,
                overlap_tokens=args.overlap_tokens
            )
            logging.info(f"[Indexing] doc_id={doc_id}, {len(chunks)} chunks")
            for i, chunk in enumerate(chunks):
                print(f"--- Chunk {i} ---")
                print(chunk)
                print("----------------")
            insert_chunks_to_milvus(collection, doc_id, chunks, batch_size=20)
            logging.info(f"[Indexing] doc_id={doc_id} hoàn thành.")
        logging.info("[Indexing] Hoàn thành indexing toàn bộ tài liệu.")
    if args.mode in {"chat", "index_and_chat"}:
        if collection is None:
            if args.collection_name not in [c.name for c in utility.list_collections()]:
                logging.error(f"Collection '{args.collection_name}' không tồn tại. Hãy chạy với --mode index trước.")
                sys.exit(1)
            collection = Collection(args.collection_name)
            collection.load()
            logging.info(f"[Milvus] Loaded existing collection '{args.collection_name}'.")
        interactive_cli(collection)

if __name__ == "__main__":
    main()